# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Dataset

In [7]:
# Load dataset 
data = pd.read_excel('Project3_DistancesMatrix.xlsx', header=None) # header=None because the first row is not a header

# Verify the dataset has missung values
print(data.isnull().sum().sum())




0


# Models

In [8]:
# Parameters
POPULATION_SIZE = 100
GENOME_LENGTH = 10
MUTATION_RATE = 0.01
CROSSOVER_RATE = 0.7
MAX_GENERATIONS = 50

In [9]:
# Fitness function
def fitness(genome):
    x = genome_to_x(genome)
    return x**2

# Convert binary string to decimal value of x
def genome_to_x(genome):
    max_val = 2**GENOME_LENGTH - 1
    return -5 + genome * 10 / max_val

# Genetic operators
def mutate(genome):
    for i in range(GENOME_LENGTH):
        if np.random.random() < MUTATION_RATE:
            genome ^= (1 << i)  # Flip the bit at position i
    return genome

def crossover(parent1, parent2):
    if np.random.random() < CROSSOVER_RATE:
        crossover_point = np.random.randint(0, GENOME_LENGTH)
        mask = (1 << crossover_point) - 1  # Mask to perform crossover
        temp = parent1
        parent1 = (parent1 & mask) | (parent2 & ~mask)
        parent2 = (temp & ~mask) | (parent2 & mask)
    return parent1, parent2

# Create initial population
population = np.random.randint(0, 2**GENOME_LENGTH, POPULATION_SIZE)

# Main loop
for generation in range(MAX_GENERATIONS):
    # Compute fitness values
    fitness_values = np.array([fitness(g) for g in population])

    # Select parents (proportional to fitness)
    parents_indices = np.random.choice(
        POPULATION_SIZE, size=(POPULATION_SIZE, 2),
        p=fitness_values/np.sum(fitness_values)
    )

    # Create next generation
    next_generation = []
    for i in range(POPULATION_SIZE // 2):
        parent1, parent2 = population[parents_indices[i]]
        offspring1, offspring2 = crossover(parent1, parent2)
        next_generation.extend([mutate(offspring1), mutate(offspring2)])
    
    population = np.array(next_generation)

# Find and print best solution
best_genome = population[np.argmax(fitness_values)]
best_x = genome_to_x(best_genome)
print(f"Best solution: x = {best_x}, f(x) = {fitness(best_genome)}")

Best solution: x = -4.853372434017595, f(x) = 23.555223983281877
